In [14]:
import os
import glob
import cv2
import datetime
import time
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import cv2
from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split, StratifiedKFold
import tensorflow
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from keras.optimizers import Adam, SGD
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
import numpy as np
from IPython.display import Image
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau 
from keras.utils import np_utils
from sklearn.metrics import log_loss

In [15]:
def create_Model():
    imageSize = 220
    model = Sequential()
    model.add(Conv2D(64, (5, 5), padding='same', activation='relu', input_shape=(imageSize, imageSize, 3)))
    model.add(Conv2D(64, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.25))
 
    model.add(Conv2D(64, (5, 5), padding='same', activation='relu'))
    model.add(Conv2D(64, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.25))
 
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.6))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.6))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy')

    
    model.summary()
    return model

In [16]:
def scr(img):
    answer = choice(['yes', 'no'])
    print(answer)
    if(answer == 'yes'):
        b,g,r = cv2.split(img)
        #print("bgr",b,g,r)
        color = [b,g,r]
        #print("color",color)
        #color2 = color.copy()
        shuffle(color)
        #print("color2",color)
        im = cv2.merge(color)
        #im = cv2.resize(im, (150,150))/255.
        #code = "1"
        #p = True
        return im
    else :
        return img



train_datagen=ImageDataGenerator(rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      channel_shift_range=0.4,                     
      preprocessing_function=scr,
      fill_mode='nearest') #included in our dependencies

train_generator=train_datagen.flow_from_directory('/home/himani/MobileNet/profile/train/',
                                                 target_size=(220,220),
                                                 color_mode='rgb',
                                                 batch_size=10,
                                                 class_mode='binary',shuffle=True)



Found 71 images belonging to 2 classes.


In [17]:
def get_im_cv2(path):
    img = cv2.imread(path)
    #print(img.shape)
    resized = cv2.resize(img, (220, 220), cv2.INTER_LINEAR)
    return resized


In [18]:
def load_train():
    X_train = []
    X_train_id = []
    y_train = []
    start_time = time.time()

    print('Read train images')
    #folders = ['0', '1']
    folders = ['human', 'none']
    for fld in folders:
        index = folders.index(fld)
        print('Load folder {} (Index: {})'.format(fld, index))
        #path = os.path.join('..', 'Coke', 'emotions', 'emotion_train', fld, '*.jpg')
        path = os.path.join('..', 'MobileNet', 'profile', 'train', fld, '*.jpg')
        #path = os.path.join('..', 'MobileNet', 'profile2', 'train', fld, '*.jpg')
        print(path)
        files = glob.glob(path)
        print(path)
        for fl in files:
            flbase = os.path.basename(fl)
            img = get_im_cv2(fl)
            X_train.append(img)
            X_train_id.append(flbase)
            y_train.append(index)

    print('Read train data time: {} seconds'.format(round(time.time() - start_time, 2)))
    return X_train, y_train, X_train_id


In [19]:
#load_train()

In [20]:
def load_test():
    #path = os.path.join('..', 'Coke', 'emotions', 'test_emotion','*.jpg')
    path = os.path.join('..', 'MobileNet', 'profile', 'test', '*.jpg')
    files = sorted(glob.glob(path))

    X_test = []
    X_test_id = []
    for fl in files:
        flbase = os.path.basename(fl)
        img = get_im_cv2(fl)
        X_test.append(img)
        X_test_id.append(flbase)

    return X_test, X_test_id

In [21]:
#load_test()

In [22]:
def create_submission(predictions, test_id, info):
    result1 = pd.DataFrame(predictions, columns=['0', '1'])
    result1.loc[:, 'image'] = pd.Series(test_id, index=result1.index)
    now = datetime.datetime.now()
    sub_file = 'submission_' + info + '_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
    result1.to_csv(sub_file, index=False)

In [23]:
def read_and_normalize_train_data():
    train_data, train_target, train_id = load_train()

    print('Convert to numpy...')
    train_data = np.array(train_data, dtype=np.uint8)
    train_target = np.array(train_target, dtype=np.uint8)

    print('Reshape...')
    #train_data = train_data.transpose((0, 3, 1, 2))
    train_data = train_data.transpose((0, 1, 2, 3))
    print('Convert to float...')
    train_data = train_data.astype('float32')
    train_data = train_data / 255
    train_target = np_utils.to_categorical(train_target, 2)

    print('Train shape:', train_data.shape)
    print(train_data.shape[0], 'train samples')
    return train_data, train_target, train_id


def read_and_normalize_test_data():
    start_time = time.time()
    test_data, test_id = load_test()

    test_data = np.array(test_data, dtype=np.uint8)
    #test_data = test_data.transpose((0, 3, 1, 2))
    test_data = test_data.transpose((0, 1, 2, 3))
    test_data = test_data.astype('float32')
    test_data = test_data / 255

    print('Test shape:', test_data.shape)
    print(test_data.shape[0], 'test samples')
    print('Read and process test data time: {} seconds'.format(round(time.time() - start_time, 2)))
    return test_data, test_id


In [24]:
def get_validation_predictions(train_data, predictions_valid):
    pv = []
    for i in range(len(train_data)):
        pv.append(predictions_valid[i])
    return pv

In [25]:
def run_cross_validation_create_models(nfolds=10):
    # input image dimensions
    batch_size = 16
    nb_epoch = 30
    random_state = 51

    train_data, train_target, train_id = read_and_normalize_train_data()
    #train_data, train_target, train_id = load_train()
    print(len(train_id))
    yfull_train = dict()
    kf = KFold(len(train_id), n_folds=nfolds, shuffle=True, random_state=random_state)
    num_fold = 0
    sum_score = 0
    models = []
    for train_index, test_index in kf:
        model = create_Model()
        '''X_train = np.array(train_data)[train_index]
        Y_train = np.array(train_target)[train_index]
        X_valid = np.array(train_data)[test_index]
        Y_valid = np.array(train_target)[test_index]'''

        X_train = train_data[train_index]
        Y_train = train_target[train_index]
        X_valid = train_data[test_index]
        Y_valid = train_target[test_index]
        
        print(" x shape ", np.shape(X_train))
        print(" y shape ", np.shape(Y_train))
        print(" x length ", len(X_train))
        print(" y length ", len(Y_train))
        
        num_fold += 1
        print('Start KFold number {} from {}'.format(num_fold, nfolds))
        print('Split train: ', len(X_train), len(Y_train))
        print('Split valid: ', len(X_valid), len(Y_valid))
        
        #model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        
        callbacks = [
            EarlyStopping(monitor='val_loss', patience=3, verbose=0),
        ]
        #model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch, shuffle=True, verbose=1, validation_data=(X_valid, Y_valid))
        model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch, shuffle=True, verbose=2, validation_data=(X_valid, Y_valid),
              callbacks=callbacks) # callbacks called for earlystoping if validation loss is not decreasing any more 

        
        
        predictions_valid = model.predict(X_valid.astype('float32'), batch_size=batch_size, verbose=2)
        score = log_loss(Y_valid, predictions_valid)
        print('Score log_loss: ', score)
        sum_score += score*len(test_index)

        # Store valid predictions
        for i in range(len(test_index)):
            yfull_train[test_index[i]] = predictions_valid[i]

        models.append(model)

    score = sum_score/len(train_data)
    print("Log_loss train independent avg: ", score)

    info_string = 'loss_' + str(score) + '_folds_' + str(nfolds) + '_ep_' + str(nb_epoch)
    return info_string, models


def run_cross_validation_process_test(info_string, models):
    batch_size = 16
    num_fold = 0
    yfull_test = []
    test_id = []
    nfolds = len(models)

    for i in range(nfolds):
        model = models[i]
        num_fold += 1
        print('Start KFold number {} from {}'.format(num_fold, nfolds))
        test_data, test_id = read_and_normalize_test_data()
        #test_data, test_id = load_test()
        test_prediction = model.predict(test_data, batch_size=batch_size, verbose=2)
        yfull_test.append(test_prediction)

    test_res = merge_several_folds_mean(yfull_test, nfolds)
    info_string = 'loss_' + info_string \
                + '_folds_' + str(nfolds)
    create_submission(test_res, test_id, info_string)


In [26]:
num_folds = 3
info_string, models = run_cross_validation_create_models(num_folds)
run_cross_validation_process_test(info_string, models)

Read train images
Load folder human (Index: 0)
../MobileNet/profile/train/human/*.jpg
../MobileNet/profile/train/human/*.jpg
Load folder none (Index: 1)
../MobileNet/profile/train/none/*.jpg
../MobileNet/profile/train/none/*.jpg
Read train data time: 0.3 seconds
Convert to numpy...
Reshape...
Convert to float...
Train shape: (29, 220, 220, 3)
29 train samples
29
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 220, 220, 64)      4864      
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 216, 216, 64)      102464    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 108, 108, 64)      0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 108, 108, 64)      102464    
_________________________________________

NameError: name 'merge_several_folds_mean' is not defined